In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json 

### Загрузка необходимых данных 

In [5]:
with open("data/test_object_lu.json", "r") as file:
    map_objects = json.load(file)
    
partition = cv2.imread('data/partition.jpg')
v_corners = np.load("data/v_corners.npy")
h_corners = np.load("data/h_corners.npy")
panorama = cv2.imread("data/image_1024_aligned_rgb.png")
depth = np.load("data/depth_map.npy") * 1000 # in millimeters 

In [6]:
print(depth.shape)
print(depth.max())
print(depth)
# Нормализация данных с камеры глубины
scaled_depth = ((depth - np.min(depth)) / (np.max(depth) - np.min(depth)) * 255).astype(np.uint8)
cv2.imshow('Depth Image', scaled_depth)
cv2.waitKey(0)
cv2.destroyAllWindows()

(512, 1024)
6101.31
[[1214.8892 1214.8892 1214.8892 ... 1214.8892 1214.8892 1214.8892]
 [1214.9348 1214.9348 1214.9348 ... 1214.9348 1214.9348 1214.9348]
 [1215.0264 1215.0264 1215.0264 ... 1215.0264 1215.0264 1215.0264]
 ...
 [1600.1882 1600.1882 1600.1882 ... 1600.1882 1600.1882 1600.1882]
 [1600.0677 1600.0677 1600.0677 ... 1600.0677 1600.0677 1600.0677]
 [1600.0076 1600.0076 1600.0076 ... 1600.0076 1600.0076 1600.0076]]


In [98]:
# Загрузить изображения
alpha = 0.7

# Преобразовать глубину в трехканальное изображение
depth_colored = cv2.applyColorMap((scaled_depth * 255).astype(np.uint8), cv2.COLORMAP_JET)

# Наложить изображения
overlay = cv2.addWeighted(panorama, alpha, depth_colored, 1 - alpha, 0)

# Вывести результат
cv2.imshow('Overlay', overlay)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [5]:
cv2.imshow("source_image", panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()
print(v_corners)

[[  30.914364  222.41164 ]
 [  30.914364  298.64883 ]
 [ 363.15613   140.81924 ]
 [ 363.15613   392.54047 ]
 [ 672.78357   149.66058 ]
 [ 672.78357   383.4991  ]
 [ 801.5895    137.09825 ]
 [ 801.5895    396.26495 ]
 [ 845.40314    76.09795 ]
 [ 845.40314   451.50623 ]
 [1008.1279    221.95033 ]
 [1008.1279    299.23853 ]]


In [154]:
start = int(v_corners[0][0])
print("start of the line:", start)
end = int(v_corners[2][0])
print("end of the line:", end)
line = []
for x in range(start, end+1, 32):
    line.append([x, h_corners[:, x][0]])
print(line)

start of the line: 30
end of the line: 363
[[30, 222.71733], [62, 192.55711], [94, 169.15553], [126, 150.01512], [158, 136.72841], [190, 129.05185], [222, 124.01126], [254, 122.649155], [286, 123.07381], [318, 126.99274], [350, 135.25735]]


In [6]:
for i in range(0, len(v_corners) - 2, 2):
    print(v_corners[i], v_corners[i + 1], v_corners[i + 2], v_corners[i + 3])

[ 30.914364 222.41164 ] [ 30.914364 298.64883 ] [363.15613 140.81924] [363.15613 392.54047]
[363.15613 140.81924] [363.15613 392.54047] [672.78357 149.66058] [672.78357 383.4991 ]
[672.78357 149.66058] [672.78357 383.4991 ] [801.5895  137.09825] [801.5895  396.26495]
[801.5895  137.09825] [801.5895  396.26495] [845.40314  76.09795] [845.40314 451.50623]
[845.40314  76.09795] [845.40314 451.50623] [1008.1279   221.95033] [1008.1279   299.23853]


In [7]:
print(h_corners)

[[221.67708 221.43863 221.38431 ... 222.38773 223.78665 223.0257 ]
 [299.86584 298.79175 299.72705 ... 298.76703 297.46896 299.3693 ]]


In [8]:
print(panorama.shape)

(512, 1024, 3)


### Влоб

In [9]:
num_rows = 250
num_columns = 1
channels = 3  # для цветных изображений (BGR)
new_image = np.zeros((num_rows, num_columns, channels), dtype=np.uint8)
for i in range(h_corners.shape[1]):
    lower, upper = h_corners[:,i].astype(int)
    sliced = panorama[:, i][lower:upper]
    resized = cv2.resize(sliced, (3, 250)).reshape((250, 1, 3))
    new_image = np.hstack([new_image, resized])
    
cv2.imshow("new_image", new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


### Сначала убираем вертикальное искажение, затем коррекция перспективы

In [10]:
pts_plane = [[[0, 0], [0, 250], [602, 0], [602, 250]],
             [[0, 0], [0, 250], [205, 0], [205, 250]],
             [[0, 0], [0, 250], [164, 0], [164, 250]],
             [[0, 0], [0, 250], [40, 0], [40, 250]],
             [[0, 0], [0, 250], [438, 0], [438, 250]]]

for i in range(0, len(v_corners) - 2, 2):
    area_idx = i // 2
    # Координаты для ключевых точек областей на исходной панораме
    left_lower, left_upper, right_lower, right_upper = v_corners[i], v_corners[i + 1], v_corners[i + 2], v_corners[i + 3]
    # Нахождение новой высоты для каждого столбца области 
    v_corners = v_corners.astype(int)
    # Ширина области на исходной панораме
    width = int(abs(left_upper[0] - right_upper[0]))
    upper_sequence = np.linspace(left_upper[1], right_upper[1], width).astype(int)
    lower_sequence = np.linspace(left_lower[1], right_lower[1], width).astype(int)
    sequence = upper_sequence - lower_sequence
    
    panorama_copy = panorama.copy()
    for new_size, panorama_pos, seq_pos in zip(sequence.astype(int), range(left_lower[0], right_lower[0]), range(width)):
        lower, upper = h_corners[:,panorama_pos].astype(int)
        sliced = panorama_copy[:, panorama_pos][lower:upper]
        resized = cv2.resize(sliced, (3, new_size))
        panorama_copy[:, panorama_pos][lower_sequence[seq_pos]:upper_sequence[seq_pos]] = resized
        
    # Отображение результата
    cv2.imshow('new_panorama', panorama_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Определение ключевых точек на панораме и плоскости
    pts_panorama = np.float32([left_lower, left_upper, right_lower, right_upper])
    
    # Вычисление матрицы преобразования
    matrix = cv2.getPerspectiveTransform(pts_panorama, np.float32(pts_plane[area_idx]))
    
    # Применение преобразования
    result = cv2.warpPerspective(panorama_copy, matrix, [pts_plane[area_idx][2][0], 250])
    
    # Отображение результата
    cv2.imshow('Projected Image', result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

TypeError: 'numpy.float32' object cannot be interpreted as an integer

In [ ]:
concatenated_panorama = np.concatenate((partition, partition), axis=1)
cv2.imshow("Concatenated", concatenated_panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def depth_to_coordinates(depth_map, fx, fy, cx, cy):
    """
    Преобразует карту глубины в 3D координаты.

    Параметры:
    - depth_map: Карта глубины.
    - fx, fy: Фокусные расстояния камеры.
    - cx, cy: Координаты центра изображения.

    Возвращает массив 3D координат в мировой системе координат.
    """
    rows, cols = depth_map.shape
    coordinates = np.zeros((rows, cols, 3), dtype=np.float32)

    for y in range(rows):
        for x in range(cols):
            depth = depth_map[y, x]
            coordinates[y, x, 0] = (x - cx) * depth / fx
            coordinates[y, x, 1] = (y - cy) * depth / fy
            coordinates[y, x, 2] = depth

    return coordinates

 # Загрузка карты глубины
depth_map = depth

# Параметры камеры (пример значения)
fx = 500.0  # фокусное расстояние по горизонтали
fy = 500.0  # фокусное расстояние по вертикали
cx = depth_map.shape[1] / 2.0  # координата центра изображения по горизонтали
cy = depth_map.shape[0] / 2.0  # координата центра изображения по вертикали

# Преобразование карты глубины в 3D координаты
coordinates = depth_to_coordinates(depth_map, fx, fy, cx, cy)

# Вывод 3D координат первой точки
print("3D Coordinates of the first point:", coordinates)



In [ ]:
import open3d as o3d
import numpy as np

# Создание случайного облака точек для примера
np.random.seed(123)
points = np.random.rand(100, 3)

# Создание объекта облака точек
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)

# Визуализация облака точек
o3d.visualization.draw_geometries([point_cloud])
# Сохранение облака точек
o3d.io.write_point_cloud("point_cloud.ply", point_cloud)

# Загрузка облака точек
loaded_point_cloud = o3d.io.read_point_cloud("point_cloud.ply")
o3d.visualization.draw_geometries([loaded_point_cloud])


In [91]:
# Получение ширины и высоты изображения
height, width, _ = panorama.shape

# Разбиение изображения на 6 частей по горизонтали
num_parts = 6
part_width = width // num_parts

# Массив для хранения частей изображения
image_parts = []
depth_parts = []
for i in range(num_parts):
    # Вычисление координат для текущей части
    start_x = i * part_width
    end_x = (i + 1) * part_width

    # Добавление текущей части в массив
    image_parts.append(panorama[:, start_x:end_x, :])
    depth_parts.append(depth[:, start_x:end_x])

# Теперь у вас есть массив image_parts, содержащий 6 частей изображения,
# к которым можно обращаться по индексам, например, image_parts[0] для первой части.

# Пример вывода каждой части для проверки
for i, part in enumerate(image_parts):
    cv2.imshow(f'Part {i+1}', part)
    cv2.imshow(f'Part {i+1}', depth_parts[i])

cv2.waitKey(0)
cv2.destroyAllWindows()

In [141]:
import open3d as o3d
import numpy as np
import cv2

# Получение ширины изображения
# Взятие половины изображения по горизонтали
rgb_image = panorama
depth_image = depth
width = rgb_image.shape[1]
# Создание облака точек из данных камеры RGB-D
rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d.geometry.Image(rgb_image), 
    o3d.geometry.Image(depth_image)
)
fx = 0.6  # фокусное расстояние по горизонтали
fy = 0.6  # фокусное расстояние по вертикали
cx = depth_image.shape[1] / 2.0  # координата центра изображения по горизонтали
cy = depth_image.shape[0] / 2.0  # координата центра изображения по вертикали

intrinsics = o3d.camera.PinholeCameraIntrinsic(
    width=rgb_image.shape[1], height=rgb_image.shape[0],
    fx=fx, fy=fy, cx=cx, cy=cy
)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
    rgbd_image,
    intrinsics)
# Flip it, otherwise the pointcloud will be upside down
pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
# Визуализация облака точек
o3d.visualization.draw_geometries([pcd])

In [124]:
# outliers removal
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=20.0)
pcd = pcd.select_by_index(ind)

# estimate normals
pcd.estimate_normals()
pcd.orient_normals_to_align_with_direction()

# surface reconstruction
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=10, n_threads=1)[0]

# rotate the mesh
rotation = mesh.get_rotation_matrix_from_xyz((np.pi, 0, 0))
mesh.rotate(rotation, center=(0, 0, 0))

# save the mesh
o3d.io.write_triangle_mesh(f'./mesh.obj', mesh)
o3d.visualization.draw_geometries([pcd])

In [122]:
import cv2
import numpy as np

# Load and prepare BMP texture
bmp = panorama

# 180x90 degree
xs, ys = bmp.shape[1], bmp.shape[0]
da = np.pi / (xs - 1)
db = 0.5 * np.pi / (ys - 1)
b = -0.25 * np.pi

# Process all pixels
pnt = []
for y in range(ys):
    for x in range(xs):
        # Pixel intensity from texture
        r = bmp[y, x] / 255.0
        # Convert to 3D
        xx = r * np.cos(x * da) * np.cos(b)
        yy = r * np.sin(x * da) * np.cos(b)
        zz = r * np.sin(b)
        # Store in point cloud
        pnt.extend([xx, yy, zz])

# Reshape the array to Nx3
point_cloud = np.array(pnt).reshape(-1, 3)

# Convert NumPy array to Open3D point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

In [179]:
import cv2
import os

image = panorama

# Получение размеров изображения
height, width = image.shape[:2]

# Разделение изображения на 6 частей по горизонтали
num_parts = 3
part_width = width // num_parts

# Создание папки для частей, если ее нет
output_folder = 'parts'
os.makedirs(output_folder, exist_ok=True)

# Сохранение каждой части изображения
for i in range(num_parts):
    start_col = i * part_width
    end_col = (i + 1) * part_width

    # Выделение части изображения
    image_part = image[:, start_col:end_col, :]

    # Сохранение части изображения
    part_filename = f"{output_folder}/part{i + 1}.jpg"
    cv2.imwrite(part_filename, image_part)

    print(f"Saved part {i + 1} to {part_filename}")


Saved part 1 to parts/part1.jpg
Saved part 2 to parts/part2.jpg
Saved part 3 to parts/part3.jpg


In [137]:
import cv2
import os

# Загрузка изображения
image_path = 'data/image_1024_aligned_rgb.png'
image = cv2.imread(image_path)

# Размер паддинга
padding_size = 200

# Добавление паддинга


# Получение размеров изображения с паддингом
height, width = image.shape[:2]

# Разделение изображения на 6 частей по горизонтали
num_parts = 6
part_width = width // num_parts

# Создание папки для частей, если ее нет
output_folder = 'parts_with_padding'
os.makedirs(output_folder, exist_ok=True)

# Сохранение каждой части изображения
for i in range(num_parts):
    start_col = i * part_width
    end_col = (i + 1) * part_width

    # Выделение части изображения с паддингом
    image_part = image[:, start_col:end_col, :]
    image_with_padding = cv2.copyMakeBorder(image_part, padding_size, padding_size, padding_size, padding_size, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    # Сохранение части изображения
    part_filename = f"{output_folder}/part{i + 1}.jpg"
    cv2.imwrite(part_filename, image_with_padding)

    print(f"Saved part {i + 1} to {part_filename}")

Saved part 1 to parts_with_padding/part1.jpg
Saved part 2 to parts_with_padding/part2.jpg
Saved part 3 to parts_with_padding/part3.jpg
Saved part 4 to parts_with_padding/part4.jpg
Saved part 5 to parts_with_padding/part5.jpg
Saved part 6 to parts_with_padding/part6.jpg


In [148]:
from defisheye import Defisheye
import cv2 

dtype = 'linear'
format = 'fullframe'
fov = 180
pfov = 120


img = "parts_with_padding/part1.jpg"

obj = Defisheye(img, dtype=dtype, format=format, fov=fov, pfov=pfov)

# To use the converted image in memory

new_image = obj.convert()

cv2.imshow("1", new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [100]:
import cv2
import numpy as np

# Функция обработки изображения с устранением barrel distortion
def undistort_image(image, k1, k2, k3):
    h, w = image.shape[:2]
    K = np.eye(3)
    
    # Коэффициенты искажения
    dist_coeffs = np.array([k1, k2, k3, 0.0, 0.0])
    
    # Устранение barrel distortion
    undistorted_image = cv2.undistort(image, K, dist_coeffs)
    
    return undistorted_image

# Функция обратного вызова для ползунков
def on_trackbar_change(val):
    # Получение текущих положений ползунков
    k1 = (cv2.getTrackbarPos('K1', 'Undistorted Image') - 1000) / 1000000000.0  # Ограничиваем диапазон от -1.0 до 1.0
    k2 = (cv2.getTrackbarPos('K2', 'Undistorted Image') - 1000) / 1000000000000000.0  # Ограничиваем диапазон от -1.0 до 1.0
    k3 = (cv2.getTrackbarPos('K3', 'Undistorted Image') - 1000) / 10000000  # Ограничиваем диапазон от -1.0 до 1.0

    # Применение устранения barrel distortion
    undistorted = undistort_image(original_image, k1, k2, k3)
    
    # Отображение обработанного изображения
    cv2.imshow('Undistorted Image', undistorted)

# Загрузка изображения
image_path = "parts_with_padding/part1.jpg"
original_image = cv2.imread(image_path)

# Создание окна
cv2.namedWindow('Undistorted Image')

# Создание ползунков
cv2.createTrackbar('K1', 'Undistorted Image', 1000, 2000, on_trackbar_change)
cv2.createTrackbar('K2', 'Undistorted Image', 1000, 2000, on_trackbar_change)
cv2.createTrackbar('K3', 'Undistorted Image', 1000, 2000, on_trackbar_change)

# Инициализация положений ползунков
cv2.setTrackbarPos('K1', 'Undistorted Image', 1000)
cv2.setTrackbarPos('K2', 'Undistorted Image', 1000)
cv2.setTrackbarPos('K3', 'Undistorted Image', 1000)

# Инициализация изображения с устранением искажения при старте
on_trackbar_change(0)

# Ожидание нажатия клавиши
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:866: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


In [107]:
import cv2
import numpy as np

def stretch_image(image, stretch_factor):
    h, w = image.shape[:2]
    
    # Создание матрицы преобразования для растяжения
    transformation_matrix = np.array([[stretch_factor, 0, 0], [0, 1, 0]], dtype=np.float32)
    
    # Применение преобразования к изображению
    stretched_image = cv2.warpAffine(image, transformation_matrix, (w, h))
    
    return stretched_image

# Загрузка изображения
image_path = "parts_with_padding/part1.jpg"
original_image = cv2.imread(image_path)

# Установка коэффициента растяжения (замените его на необходимое значение)
stretch_factor = 4.5

# Применение растяжения
stretched_image = stretch_image(original_image, stretch_factor)

# Отображение изображений
cv2.imshow('Original Image', original_image)
cv2.imshow('Stretched Image', stretched_image)

# Ожидание нажатия клавиши
cv2.waitKey(0)
cv2.destroyAllWindows()


In [16]:
import cv2
import open3d as o3d
import numpy as np

def depth_to_pointcloud(depth_image, fx, fy, cx, cy):
    # Получение координат точек из изображения глубины
    rows, cols = depth_image.shape
    y, x = np.indices((rows, cols))
    z = depth_image / 1000.0  # Перевод из миллиметров в метры

    # Конвертация в координаты в 3D пространстве
    x3d = (x - cx) * z / fx
    y3d = (y - cy) * z / fy

    # Создание облака точек
    points = np.column_stack((x3d.flatten(), y3d.flatten(), z.flatten()))
    valid_points = points[np.isfinite(points).all(axis=1)]

    return valid_points

# Загрузка изображения глубины
width = scaled_depth.shape[1]
depth_image = scaled_depth
# Отображение изображений
cv2.imshow('Original Image', depth_image)

# Ожидание нажатия клавиши
cv2.waitKey(0)
cv2.destroyAllWindows()
# Калибровочные параметры камеры (замените их на ваши значения)
fx = 550.0
fy = 100.0
cx = depth_image.shape[1] / 2
cy = depth_image.shape[0] / 2

# Преобразование изображения глубины в PointCloud
pointcloud = depth_to_pointcloud(depth_image, fx, fy, cx, cy)

# Создание объекта PointCloud в Open3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pointcloud)

# Визуализация PointCloud
o3d.visualization.draw_geometries([pcd])


In [19]:
import numpy as np
from scipy.ndimage import map_coordinates


def xyzcube(face_w):
    '''
    Return the xyz cordinates of the unit cube in [F R B L U D] format.
    '''
    out = np.zeros((face_w, face_w * 6, 3), np.float32)
    rng = np.linspace(-0.5, 0.5, num=face_w, dtype=np.float32)
    grid = np.stack(np.meshgrid(rng, -rng), -1)

    # Front face (z = 0.5)
    out[:, 0*face_w:1*face_w, [0, 1]] = grid
    out[:, 0*face_w:1*face_w, 2] = 0.5

    # Right face (x = 0.5)
    out[:, 1*face_w:2*face_w, [2, 1]] = grid
    out[:, 1*face_w:2*face_w, 0] = 0.5

    # Back face (z = -0.5)
    out[:, 2*face_w:3*face_w, [0, 1]] = grid
    out[:, 2*face_w:3*face_w, 2] = -0.5

    # Left face (x = -0.5)
    out[:, 3*face_w:4*face_w, [2, 1]] = grid
    out[:, 3*face_w:4*face_w, 0] = -0.5

    # Up face (y = 0.5)
    out[:, 4*face_w:5*face_w, [0, 2]] = grid
    out[:, 4*face_w:5*face_w, 1] = 0.5

    # Down face (y = -0.5)
    out[:, 5*face_w:6*face_w, [0, 2]] = grid
    out[:, 5*face_w:6*face_w, 1] = -0.5

    return out


def equirect_uvgrid(h, w):
    u = np.linspace(-np.pi, np.pi, num=w, dtype=np.float32)
    v = np.linspace(np.pi, -np.pi, num=h, dtype=np.float32) / 2

    return np.stack(np.meshgrid(u, v), axis=-1)


def equirect_facetype(h, w):
    '''
    0F 1R 2B 3L 4U 5D
    '''
    tp = np.roll(np.arange(4).repeat(w // 4)[None, :].repeat(h, 0), 3 * w // 8, 1)

    # Prepare ceil mask
    mask = np.zeros((h, w // 4), np.bool)
    idx = np.linspace(-np.pi, np.pi, w // 4) / 4
    idx = h // 2 - np.round(np.arctan(np.cos(idx)) * h / np.pi).astype(int)
    for i, j in enumerate(idx):
        mask[:j, i] = 1
    mask = np.roll(np.concatenate([mask] * 4, 1), 3 * w // 8, 1)

    tp[mask] = 4
    tp[np.flip(mask, 0)] = 5

    return tp.astype(np.int32)


def xyzpers(h_fov, v_fov, u, v, out_hw, in_rot):
    out = np.ones((*out_hw, 3), np.float32)

    x_max = np.tan(h_fov / 2)
    y_max = np.tan(v_fov / 2)
    x_rng = np.linspace(-x_max, x_max, num=out_hw[1], dtype=np.float32)
    y_rng = np.linspace(-y_max, y_max, num=out_hw[0], dtype=np.float32)
    out[..., :2] = np.stack(np.meshgrid(x_rng, -y_rng), -1)
    Rx = rotation_matrix(v, [1, 0, 0])
    Ry = rotation_matrix(u, [0, 1, 0])
    Ri = rotation_matrix(in_rot, np.array([0, 0, 1.0]).dot(Rx).dot(Ry))

    return out.dot(Rx).dot(Ry).dot(Ri)


def xyz2uv(xyz):
    '''
    xyz: ndarray in shape of [..., 3]
    '''
    x, y, z = np.split(xyz, 3, axis=-1)
    u = np.arctan2(x, z)
    c = np.sqrt(x**2 + z**2)
    v = np.arctan2(y, c)

    return np.concatenate([u, v], axis=-1)


def uv2unitxyz(uv):
    u, v = np.split(uv, 2, axis=-1)
    y = np.sin(v)
    c = np.cos(v)
    x = c * np.sin(u)
    z = c * np.cos(u)

    return np.concatenate([x, y, z], axis=-1)


def uv2coor(uv, h, w):
    '''
    uv: ndarray in shape of [..., 2]
    h: int, height of the equirectangular image
    w: int, width of the equirectangular image
    '''
    u, v = np.split(uv, 2, axis=-1)
    coor_x = (u / (2 * np.pi) + 0.5) * w - 0.5
    coor_y = (-v / np.pi + 0.5) * h - 0.5

    return np.concatenate([coor_x, coor_y], axis=-1)


def coor2uv(coorxy, h, w):
    coor_x, coor_y = np.split(coorxy, 2, axis=-1)
    u = ((coor_x + 0.5) / w - 0.5) * 2 * np.pi
    v = -((coor_y + 0.5) / h - 0.5) * np.pi

    return np.concatenate([u, v], axis=-1)


def sample_equirec(e_img, coor_xy, order):
    w = e_img.shape[1]
    coor_x, coor_y = np.split(coor_xy, 2, axis=-1)
    pad_u = np.roll(e_img[[0]], w // 2, 1)
    pad_d = np.roll(e_img[[-1]], w // 2, 1)
    e_img = np.concatenate([e_img, pad_d, pad_u], 0)
    return map_coordinates(e_img, [coor_y, coor_x],
                           order=order, mode='wrap')[..., 0]


def sample_cubefaces(cube_faces, tp, coor_y, coor_x, order):
    cube_faces = cube_faces.copy()
    cube_faces[1] = np.flip(cube_faces[1], 1)
    cube_faces[2] = np.flip(cube_faces[2], 1)
    cube_faces[4] = np.flip(cube_faces[4], 0)

    # Pad up down
    pad_ud = np.zeros((6, 2, cube_faces.shape[2]))
    pad_ud[0, 0] = cube_faces[5, 0, :]
    pad_ud[0, 1] = cube_faces[4, -1, :]
    pad_ud[1, 0] = cube_faces[5, :, -1]
    pad_ud[1, 1] = cube_faces[4, ::-1, -1]
    pad_ud[2, 0] = cube_faces[5, -1, ::-1]
    pad_ud[2, 1] = cube_faces[4, 0, ::-1]
    pad_ud[3, 0] = cube_faces[5, ::-1, 0]
    pad_ud[3, 1] = cube_faces[4, :, 0]
    pad_ud[4, 0] = cube_faces[0, 0, :]
    pad_ud[4, 1] = cube_faces[2, 0, ::-1]
    pad_ud[5, 0] = cube_faces[2, -1, ::-1]
    pad_ud[5, 1] = cube_faces[0, -1, :]
    cube_faces = np.concatenate([cube_faces, pad_ud], 1)

    # Pad left right
    pad_lr = np.zeros((6, cube_faces.shape[1], 2))
    pad_lr[0, :, 0] = cube_faces[1, :, 0]
    pad_lr[0, :, 1] = cube_faces[3, :, -1]
    pad_lr[1, :, 0] = cube_faces[2, :, 0]
    pad_lr[1, :, 1] = cube_faces[0, :, -1]
    pad_lr[2, :, 0] = cube_faces[3, :, 0]
    pad_lr[2, :, 1] = cube_faces[1, :, -1]
    pad_lr[3, :, 0] = cube_faces[0, :, 0]
    pad_lr[3, :, 1] = cube_faces[2, :, -1]
    pad_lr[4, 1:-1, 0] = cube_faces[1, 0, ::-1]
    pad_lr[4, 1:-1, 1] = cube_faces[3, 0, :]
    pad_lr[5, 1:-1, 0] = cube_faces[1, -2, :]
    pad_lr[5, 1:-1, 1] = cube_faces[3, -2, ::-1]
    cube_faces = np.concatenate([cube_faces, pad_lr], 2)

    return map_coordinates(cube_faces, [tp, coor_y, coor_x], order=order, mode='wrap')


def cube_h2list(cube_h):
    assert cube_h.shape[0] * 6 == cube_h.shape[1]
    return np.split(cube_h, 6, axis=1)


def cube_list2h(cube_list):
    assert len(cube_list) == 6
    assert sum(face.shape == cube_list[0].shape for face in cube_list) == 6
    return np.concatenate(cube_list, axis=1)


def cube_h2dict(cube_h):
    cube_list = cube_h2list(cube_h)
    return dict([(k, cube_list[i])
                 for i, k in enumerate(['F', 'R', 'B', 'L', 'U', 'D'])])


def cube_dict2h(cube_dict, face_k=['F', 'R', 'B', 'L', 'U', 'D']):
    assert len(face_k) == 6
    return cube_list2h([cube_dict[k] for k in face_k])


def cube_h2dice(cube_h):
    assert cube_h.shape[0] * 6 == cube_h.shape[1]
    w = cube_h.shape[0]
    cube_dice = np.zeros((w * 3, w * 4, cube_h.shape[2]), dtype=cube_h.dtype)
    cube_list = cube_h2list(cube_h)
    # Order: F R B L U D
    sxy = [(1, 1), (2, 1), (3, 1), (0, 1), (1, 0), (1, 2)]
    for i, (sx, sy) in enumerate(sxy):
        face = cube_list[i]
        if i in [1, 2]:
            face = np.flip(face, axis=1)
        if i == 4:
            face = np.flip(face, axis=0)
        cube_dice[sy*w:(sy+1)*w, sx*w:(sx+1)*w] = face
    return cube_dice


def cube_dice2h(cube_dice):
    w = cube_dice.shape[0] // 3
    assert cube_dice.shape[0] == w * 3 and cube_dice.shape[1] == w * 4
    cube_h = np.zeros((w, w * 6, cube_dice.shape[2]), dtype=cube_dice.dtype)
    # Order: F R B L U D
    sxy = [(1, 1), (2, 1), (3, 1), (0, 1), (1, 0), (1, 2)]
    for i, (sx, sy) in enumerate(sxy):
        face = cube_dice[sy*w:(sy+1)*w, sx*w:(sx+1)*w]
        if i in [1, 2]:
            face = np.flip(face, axis=1)
        if i == 4:
            face = np.flip(face, axis=0)
        cube_h[:, i*w:(i+1)*w] = face
    return cube_h


def rotation_matrix(rad, ax):
    ax = np.array(ax)
    assert len(ax.shape) == 1 and ax.shape[0] == 3
    ax = ax / np.sqrt((ax**2).sum())
    R = np.diag([np.cos(rad)] * 3)
    R = R + np.outer(ax, ax) * (1.0 - np.cos(rad))

    ax = ax * np.sin(rad)
    R = R + np.array([[0, -ax[2], ax[1]],
                      [ax[2], 0, -ax[0]],
                      [-ax[1], ax[0], 0]])

    return R

In [163]:
import numpy as np


def e2p(e_img, fov_deg, u_deg, v_deg, out_hw, in_rot_deg=0, mode='bilinear'):
    '''
    e_img:   ndarray in shape of [H, W, *]
    fov_deg: scalar or (scalar, scalar) field of view in degree
    u_deg:   horizon viewing angle in range [-180, 180]
    v_deg:   vertical viewing angle in range [-90, 90]
    '''
    assert len(e_img.shape) == 3
    h, w = e_img.shape[:2]

    try:
        h_fov, v_fov = fov_deg[0] * np.pi / 180, fov_deg[1] * np.pi / 180
    except:
        h_fov, v_fov = fov, fov
    in_rot = in_rot_deg * np.pi / 180

    if mode == 'bilinear':
        order = 1
    elif mode == 'nearest':
        order = 0
    else:
        raise NotImplementedError('unknown mode')

    u = -u_deg * np.pi / 180
    v = v_deg * np.pi / 180
    xyz = xyzpers(h_fov, v_fov, u, v, out_hw, in_rot)
    uv = xyz2uv(xyz)
    coor_xy = uv2coor(uv, h, w)

    pers_img = np.stack([
        sample_equirec(e_img[..., i], coor_xy, order=order)
        for i in range(e_img.shape[2])
    ], axis=-1)

    return pers_img


equirectangular_img = panorama
h, w = equirectangular_img.shape[:2]
print(h, w)
views = [['f', (120, 120), 0, 0, 0],
         ['r', (120, 120), 90, 0, 0],
         ['b', (120, 120), 180, 0, 0],
         ['l', (120, 120), -90, 0, 0],
         ['u', (120, 120), 0, 90, 0],
         ['d', (120, 120), 0, -90, 0]]
# Углы обзора
# fov_deg = (120, 120)
# u_deg = 0
# v_deg = 90

# Размер выходного изображения
# out_hw = (1200, 900)
for face_k, fov_deg, u_deg, v_deg, in_rot in views:
    out_hw = (
        (512 * fov_deg[1]) // 180 * 2,
        (1024 * fov_deg[0]) // 360 * 2,
    )
    # Угол поворота входного изображения
    in_rot_deg = 0
    
    # Режим интерполяции
    mode = 'bilinear'
    
    # Вызов функции e2p
    pers_img = e2p(equirectangular_img, fov_deg, u_deg, v_deg, out_hw, in_rot_deg, mode)
    bgr_scaled_depth = cv2.cvtColor(scaled_depth, cv2.COLOR_GRAY2BGR)
    pers_depth = e2p(bgr_scaled_depth, fov_deg, u_deg, v_deg, out_hw, in_rot_deg, mode)
    
    # Визуализация результата (при необходимости) 
    # cv2.imshow('Perspective Image', pers_img)
    # cv2.imshow('Perspective Depth', pers_depth)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # Сохранение изображений
    cv2.imwrite(f"frblud/{face_k}_depth.jpg", pers_depth)
    cv2.imwrite(f"frblud/{face_k}_image.jpg", pers_img)

512 1024


In [104]:
from defisheye import Defisheye
import cv2 

dtype = 'linear'
format = 'fullframe'
fov = 160
pfov = 90


img = "parts_with_padding/part1.jpg"

obj = Defisheye(pers_img, dtype=dtype, format=format, fov=fov, pfov=pfov)

# To use the converted image in memory

new_image = obj.convert()

cv2.imshow("1", new_image)
cv2.waitKey(0)
cv2.destroyAllWindows()